In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from itertools import *

In [3]:
sc = pyspark.SparkContext("local","Basket Analysis")

In [4]:
basket = sc.textFile("/content/drive/MyDrive/browsing.txt")
basket.take(5)

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ',
 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ',
 'ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ',
 'ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 ',
 'ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 ']

In [5]:
print( "Total Number of Baskets : ", basket.count())

Total Number of Baskets :  31101


In [6]:
from operator import add
itemsFrequency = basket.flatMap(lambda line:(line.split())).filter(lambda x : len(x) != 0).map(lambda x:(x,1)).reduceByKey(add)
freqItems = itemsFrequency.filter(lambda x: x[1] >=100).sortBy( lambda x : -x[1])
# freqItemsList = freqItems.map(lambda x:x[0]).collect()
print("Total Number of Items with Support Greater Than 100 : ", (freqItems.count()))
# print(freqItemsList.take(2))

Total Number of Items with Support Greater Than 100 :  647


In [19]:
# Question 01: Identify pairs of items (X, Y) such that the support of {X, Y } is at least 100. For
# all such pairs, compute the confidence scores of the corresponding association rules: X ⇒ Y ,
# Y ⇒ X. Sort the rules in decreasing order of confidence scores and list the top 5 rules in the writeup.

pairs = basket.flatMap( lambda line : combinations((line.split()),2)).map(lambda x:(x,1)).reduceByKey(add)
pairsItems = pairs.filter(lambda x: x[1]>=100).sortBy(lambda x : -x[1])
print("Total No of Pairs with frequency greater than 100 : ", pairsItems.count())

Total No of Pairs with frequency greater than 100 :  760


In [9]:
confidence = []

for y in pairsItems.toLocalIterator():
    valueX = y[0][0]
    freqX = itemsFrequency.filter(lambda x : x[0]== valueX).map(lambda x : x[1]).collect()
    confidenceX = y[1] / freqX[0]
  
    valueY = y[0][1]
    freqY = itemsFrequency.filter(lambda x : x[0]== valueY).map(lambda x : x[1]).collect()
    confidenceY = y[1] / freqY[0]

    confidence.append(((valueX, valueY),confidenceX, confidenceY))

In [16]:
maxConfidence =  sorted(confidence, key = lambda x: -x[1])
print(maxConfidence)

[(('DAI43868', 'SNA82528'), 0.9527027027027027, 0.47474747474747475), (('GRO89004', 'ELE25077'), 0.6948051948051948, 0.4476987447698745), (('ELE92920', 'DAI62779'), 0.6716791979949874, 0.12059397030148493), (('SNA18336', 'DAI62779'), 0.6516220028208745, 0.06929653517324134), (('SNA18336', 'ELE92920'), 0.6361071932299013, 0.3767752715121136), (('DAI46755', 'FRO81176'), 0.5764705882352941, 0.31343283582089554), (('GRO81647', 'GRO73461'), 0.5408163265306123, 0.07357023875624653), (('FRO47962', 'DAI75645'), 0.5392156862745098, 0.06030701754385965), (('DAI55148', 'DAI62779'), 0.4358974358974359, 0.06374681265936703), (('SNA30533', 'SNA96271'), 0.43373493975903615, 0.11119691119691119), (('ELE59028', 'DAI62779'), 0.42199108469539376, 0.04259787010649468), (('DAI83031', 'DAI94679'), 0.4175084175084175, 0.37462235649546827), (('SNA53220', 'SNA93860'), 0.41379310344827586, 0.17057569296375266), (('ELE88583', 'SNA24799'), 0.4054878048780488, 0.2638888888888889), (('ELE99737', 'DAI85309'), 0.4050

In [11]:
# Question 02: Identify item triples (X, Y, Z) such that the support of {X, Y, Z} is at least 100.
# For all such triples, compute the confidence scores of the corresponding association rules: 
# (X, Y ) ⇒ Z, (X, Z) ⇒ Y , (Y, Z) ⇒ X. Sort the rules in decreasing order of confidence scores and
# list the top 5 rules in the writeup. 

triplesItems = basket.flatMap( lambda line : combinations((line.split()),3)).map(lambda x:(x,1)).reduceByKey(add).filter(lambda x: x[1]>=100).sortBy(lambda x : -x[1])
print("Total No of Triples with frequency greater than 100 : ", triplesItems.count())
print(triplesItems.take(10))

Total No of Triples with frequency greater than 100 :  67
[(('SNA18336', 'ELE92920', 'DAI62779'), 406), (('ELE92920', 'DAI62779', 'ELE17451'), 300), (('FRO40251', 'GRO85051', 'SNA80324'), 259), (('DAI62779', 'SNA80324', 'ELE17451'), 231), (('SNA18336', 'DAI62779', 'ELE17451'), 222), (('FRO40251', 'DAI62779', 'ELE17451'), 221), (('SNA18336', 'ELE92920', 'ELE17451'), 219), (('DAI62779', 'ELE17451', 'DAI85309'), 212), (('DAI62779', 'ELE17451', 'DAI75645'), 211), (('FRO40251', 'GRO85051', 'DAI62779'), 204)]


In [25]:
confidenceXYZ = []

for y in triplesItems.toLocalIterator():
    valueX = y[0][0]
    valueY = y[0][1]
    freqX = pairs.filter(lambda x : ((x[0][0] == valueX and x[0][1] == valueY) or (x[0][0] == valueY and x[0][1] == valueX))).map(lambda x : x[1]).collect()
    confidenceX = y[1] / freqX[0]
    
    valueY = y[0][1]
    valueZ = y[0][2]
    freqY = pairs.filter(lambda x : (x[0][0] == valueY and x[0][1] == valueZ) or (x[0][0] == valueZ and x[0][1] == valueY)).map(lambda x : x[1]).collect()
    
    confidenceY = y[1] / freqY[0]

    valueX = y[0][0]
    valueZ = y[0][2]
    freqZ = pairs.filter(lambda x : (x[0][0] == valueX and x[0][1] == valueZ) or (x[0][0] == valueZ and x[0][1] == valueX)).map(lambda x : x[1]).collect()
    confidenceZ = y[1] / freqZ[0]
    
    confidenceXYZ.append(((valueX, valueY, valueZ),confidenceX, confidenceY, confidenceZ))

In [26]:
maxConfidence =  sorted(confidenceXYZ, key = lambda x: -x[1])
print(maxConfidence)

[(('GRO81087', 'ELE92920', 'DAI62779'), 0.9098360655737705, 0.13805970149253732, 0.37373737373737376), (('SNA18336', 'ELE92920', 'DAI62779'), 0.9002217294900222, 0.5049751243781094, 0.8787878787878788), (('GRO85051', 'DAI62779', 'ELE17451'), 0.8206896551724138, 0.08686131386861314, 1.7761194029850746), (('GRO73461', 'FRO47962', 'DAI75645'), 0.7573529411764706, 0.6242424242424243, 0.14992721979621543), (('DAI62779', 'SNA59903', 'ELE17451'), 0.7253521126760564, 0.7573529411764706, 0.07518248175182482), (('SNA80324', 'ELE17451', 'DAI75645'), 0.7180851063829787, 0.6887755102040817, 0.24953789279112754), (('DAI62779', 'SNA53220', 'SNA93860'), 0.6402116402116402, 0.5041666666666667, 0.3261455525606469), (('GRO85051', 'SNA80324', 'ELE17451'), 0.6130952380952381, 0.5478723404255319, 1.537313432835821), (('DAI62779', 'ELE99737', 'DAI85309'), 0.5810055865921788, 0.16938110749185667, 0.30678466076696165), (('DAI62779', 'SNA53220', 'FRO19221'), 0.5661375661375662, 0.8770491803278688, 0.39051094890